### Inventory Simulation
#### INF1036 - 3WA

Fernando Homem, Júlia Aleixo, Pedro Moraes, Rafael Rubim

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats
from ipywidgets import FloatProgress
from IPython.display import display

%matplotlib inline

#### Variable declarations

In [2]:
T = 6.0
S = 15 # maximum on-hand inventory
s = 5 # minimum on-hand inventory
x0 = 15 # initial amount of inventory
r = 50 # price per product unit
L = 2 # units of time before order arrives
h = 2 # inventory holding cost
ratemax = 10 # maximum number of customers per time unit
receiveAfterClose = True # orders can arrive after store closes
hUntilClose = True # inventory holding cost is charged until store closes
costPerUnit = 10

#### Cost function
Defines the cost of ordering y units of the product.

In [3]:
def cost(y): 
    return y * costPerUnit

#### Customer's arrivals
Customers demanding the product appear in accordance with a non-homogenous Poisson process with rate λ.

In [4]:
def rate(t):
    if (t < 1.0):
        return 10.0
    elif (t < 2.0):
        return 5.0
    elif (t < 3.0):
        return 10.0
    elif (t < 4.0):
        return 2.0
    elif (t < 5.0):
        return 5.0
    else:
        return 10.0

In [5]:
def exponential(nsamples,rate):
    x = np.zeros(nsamples)
    u = np.random.sample(nsamples)
    for i in range(nsamples):
        x[i] = - math.log(1.0 - u[i]) / rate
    return x

In [6]:
def nextArrival(t,rate,ratemax):
    while(1):
        Z = exponential(1,ratemax)[0]
        t = t + Z
        U = np.random.sample(1)
        if U < rate(t) / ratemax:
            return t

#### Inventory Model
Returns the average profit per unit of time.

In [7]:
def inventory(G,T,S,s,c,x0,r,L,h,rate,ratemax,receiveAfterClose,hUntilClose):
    
    C = 0.0 # total amount of ordering costs
    R = 0.0 # total amount of revenue earned
    H = 0.0 # total amount of inventory holding costs
    x = x0 # amount of inventory
    y = 0 # amount on order
    t = 0.0 
    t0 = nextArrival(t,rate,ratemax) # arrival time of next customer
    t1 = 1.0e+30 # time of order delivery
    T0 = []
    T1 = []
    
    while(1):
        
        # customer arrives before delivery
        if t0 < t1 and t0 <= T:
            H = H + (t0 - t) * x * h
            t = t0
            T0.append(t0)
            D = G(1,rate(t))[0] # customer's demand
            w = min(D,x) # amount of the order that can be filled
            R = R + w * r
            x = x - w
            if x < s and y == 0:
                y = S - x
                t1 = t + L
            t0 = nextArrival(t,rate,ratemax)
            
        # order arrives before next customer
        elif (t1 <= t0 or t0 > T) and t1 <= T:
            H = H + (t1 - t) * x * h
            t = t1
            T1.append(t1)
            C = C + c(y)
            x = x + y
            y = 0
            t1 = 1.0e+30
            
        else:
            # store closes and there is still an outstanding order
            if y > 0 and receiveAfterClose:
                H = H + (t1 - t) * x * h
                t = t1
                T1.append(t1)
                C = C + c(y)
                x = x + y
                y = 0
                
            elif hUntilClose:
                H = H + (T - t) * x * h
                
            totalProfit = R - C - H
            profitPerUnit = totalProfit / T # profit per unit of time
            return totalProfit, profitPerUnit, T0, T1

In [8]:
totalProfit,profitPerUnit,T0,T1 = inventory(exponential,T,S,s,cost,x0,r,L,h,rate,ratemax,receiveAfterClose,hUntilClose)
print("Total Profit: " + str(totalProfit))
print("Profit per unit of time: " + str(profitPerUnit))
print("Customer arrival times: " + str(T0))
print("Order arrival times: " + str(T1))

Total Profit: 216.5443649712434
Profit per unit of time: 36.09072749520723
Customer arrival times: [0.004140437636058718, 0.061033802852484366, 0.1545434916505422, 0.16336761996999302, 0.18828838035367987, 0.2687768550944456, 0.6161859338341331, 0.6373037815194584, 0.6985081638916735, 0.7305666051966833, 0.8435527703929411, 0.8568401391027552, 0.9616922676251464, 0.9813626543606169, 1.0178712282978781, 1.0670930070683673, 1.294757470266706, 1.4382428328265375, 1.9350885778472233, 2.005762630689831, 2.1600152711855216, 2.7256688135003806, 4.1956043105596486, 4.267293761859133, 4.382538176966529, 4.532167424590274, 4.967755288551966, 5.142845898094178, 5.168529606848759, 5.277130019310619, 5.345423227860266, 5.5025142093955965, 5.507509987659853, 5.51978412840825, 5.6038657388533775, 5.636973371229601, 5.758757486122017, 5.798299350466128, 5.834559067585878, 5.910903537851875, 5.928706655611574]
Order arrival times: []


#### Tests

In [9]:
def inventoryTolerance(tol,alpha):
    x = np.zeros(100)
    for i in range(100):
        totalProfit,profitPerUnit,T0,T1 = inventory(exponential,T,S,s,cost,x0,r,L,h,rate,ratemax,recieveAfterClose,hUntilClose)
        x[i] = profitPerUnit
    n  = 100
    m  = np.mean(x)
    s2 = np.var(x)
    zab2 = scipy.stats.norm.ppf(1.0 - alpha / 2.0)
    while(2.0 * (s2 / n) * zab2 * zab2 > tol**2):
        totalProfit,nx,T0,T1 = inventory(exponential,T,S,s,cost,x0,r,L,h,rate,ratemax,recieveAfterClose,hUntilClose)
        nm  = m + (nx - m) / (n + 1)
        ns2 = (1.0 - 1.0 / n) * s2 + (n + 1.0) * (nm - m)**2
        n = n + 1
        m  = nm
        s2 = ns2 
        x = np.append(x,nx)
    return m,s2,n,x

In [ ]:
tol = 0.1
alpha = 0.05

m,s2,n,x = inventoryTolerance(tol,alpha)
print("Average profit per unit of time: " + str(m))
print(s2)
print("Number of necessary scenarios: " + str(n))
zab2 = scipy.stats.norm.ppf(1.0 - alpha / 2.0)
print("The average is in the interval [" + str(m - math.sqrt(s2 / n) * zab2) + "," + str(m + math.sqrt(s2 / n) * zab2) \
      + "] with a probability of " + str(1.0 - alpha) + ".")

In [ ]:
plt.hist(x, 20, density = 1, facecolor ='green', alpha = 0.5)
plt.title("Profit per Unit of Time Distribution")
plt.show()

#### Ordering Policy

By varying the values of S and s, we are able to determine the ordering policy that maximizes the store's profits.

In [ ]:
f = FloatProgress(min = 0, max = 100)
display(f)
f.description = "Loading"

maximum_profit = -1.0e+30
best_S = 0
best_s = 0

for S in range(10,20):
    for s in range(20,10,-1):
        m,s2,n,x = inventoryTolerance(tol,alpha)
        if m > maximum_profit:
            maximum_profit = m
            best_S = S
            best_s = s
        f.value = f.value + 1
            
print("Maximum profit: " + str(maximum_profit))
print("Best ordering policy (s,S): (" + str(best_s) + "," + str(best_S) + ")")